In [ ]:
from flash.image import SemanticSegmentation
from flash.image import SemanticSegmentationData


dm = SemanticSegmentationData.from_folders(
    predict_folder=r"testimgs/",
    transform_kwargs=dict(image_size=(1600, 1280)),
    batch_size = 1,
    )


from flash import Trainer

import cv2
import numpy as np
import torch

: 

In [ ]:
model = SemanticSegmentation.load_from_checkpoint(r"unet/lightning_logs/version_42_mixprimadual/checkpoints/epoch=49-step=9599.ckpt")
# model = SemanticSegmentation.load_from_checkpoint(r"unet/lightning_logs/version_41_mixprimatern/checkpoints/epoch=49-step=9599.ckpt")
# model = SemanticSegmentation.load_from_checkpoint(r"unet/lightning_logs/version_43_plainprima/checkpoints/epoch=49-step=9599.ckpt")


In [22]:
trainer = Trainer(max_epochs=1, gpus = 1)
predictions = trainer.predict(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

C:\Users\pveen\AppData\Local\Programs\Python\Python36\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting:   4%|▎         | 1/28 [00:00<00:10,  2.67it/s]

C:\Users\pveen\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\nn\functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Predicting: 100%|██████████| 28/28 [00:06<00:00,  4.20it/s]


In [23]:

for i in range(len(predictions)):
    inp = (predictions[i][0]['input']).numpy()
    inp = np.transpose(inp, (1, 2, 0))/np.max(inp)
    pred = (predictions[i][0]['preds'])
    pred = torch.softmax(pred, dim = 0)
    pred = torch.argmax(pred, dim = 0).numpy()
    # print(pred.shape, inp.shape)
    inp[:, :, 0][pred == 1] = 255
    # inp[:, :, 1][pred == 0] = 50    pred = pred.astype(np.float)

    cv2.imwrite('segs/' + str(i) + '.png', (pred*255).astype(np.uint8))
    cv2.imwrite('outputs/' + str(i) + '.png', (inp*255).astype(np.uint8))
    # pred /= np.max(pred)
